# Goal

Our goal in this notebook is to take a time series and fill the gap. We will do this in 3 ways.
   
   1. Let $g_1$, $g_2$, $\cdots$, $g_5$ be the 5 gaps. $T_0$ and $T_6$ be the two data points before and after $g_1$ and $g_5$. A linear line is drawn from $T_0$ to $T_6$ and then those gaps follow that line.
   2. figure out what is time series linear regression and implement it. or, perhaps it is already.

In [1]:
import csv
import numpy as np
import pandas as pd
# import geopandas as gpd
from IPython.display import Image
# from shapely.geometry import Point, Polygon
from math import factorial
import scipy
import scipy.signal
import os, os.path

from datetime import date
import datetime
import time

from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn.linear_model import LinearRegression
from patsy import cr

# from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sb


import sys

sys.path.append('/Users/hn/Documents/00_GitHub/Ag/remote_sensing/python/')
import remote_sensing_core as rc
import remote_sensing_core as rcp


In [2]:
data_dir =  "/Users/hn/Documents/01_research_data/remote_sensing/test_data/"

In [3]:
f_name = "a_regularized_TS.csv"
a_regularized_TS = pd.read_csv(data_dir + f_name, low_memory=False)

# Parameters

In [4]:
indeks = "EVI"
SF_year = 2017

# Linear Line

In [5]:
# a_regularized_TS.head(2)

In [6]:
# A = rc.fill_theGap_linearLine(a_regularized_TS, indeks, SF_year)

In [7]:
a_regularized_TS.head(2)

,ID,Acres,county,CropGrp,CropTyp,DataSrc,ExctAcr,IntlSrD,Irrigtn,LstSrvD,Notes,RtCrpTy,Shap_Ar,Shp_Lng,TRS,image_year,SF_year,doy,EVI
0,115184_WSDA_SF_2017,72.0,Grant,Orchard,apple,wsda,72.371199,2006/08/31,micro-sprinkler,2017/04/24,NaN,NaN,292875.850894,3308.620365,T13R24E4,2016,2017,274,0.417561
1,115184_WSDA_SF_2017,72.0,Grant,Orchard,apple,wsda,72.371199,2006/08/31,micro-sprinkler,2017/04/24,NaN,NaN,292875.850894,3308.620365,T13R24E4,2016,2017,284,-2.000000


# Create Aeolus like space and test

In [8]:
a_regularized_TS_1 = a_regularized_TS.copy()
a_regularized_TS_2 = a_regularized_TS.copy()
a_regularized_TS_3 = a_regularized_TS.copy()
a_regularized_TS_4 = a_regularized_TS.copy()

a_regularized_TS_1["ID"] = "1"
a_regularized_TS_2["ID"] = "2"
a_regularized_TS_3["ID"] = "3"
a_regularized_TS_4["ID"] = "4"

In [9]:
an_EE_TS = pd.concat([a_regularized_TS_1, a_regularized_TS_2, a_regularized_TS_3, a_regularized_TS_4])

In [ ]:
polygon_list = an_EE_TS['ID'].unique()
print(len(polygon_list))

########################################################################################
###
###  initialize output data. all polygons in this case
###  will have the same length. 
###  9 steps in the first three months, followed by 36 points in the full year,
###  9 months in the last year
###
reg_cols = an_EE_TS.columns
nrows = 54 * len(polygon_list)
output_df = pd.DataFrame(data = None,
                         index = np.arange(nrows), 
                         columns = reg_cols)


In [ ]:
counter = 0

for a_poly in polygon_list:
    if (counter % 300 == 0):
        print (counter)
    curr_field = an_EE_TS[an_EE_TS['ID']==a_poly].copy()
    ################################################################
    # Sort by DoY (sanitary check)
    curr_field.sort_values(by=['image_year', 'doy'], inplace=True)
    
    curr_field.reset_index(drop=True, inplace=True)
    
    print ("print(curr_field.shape)")
    print(curr_field.shape)
    print ("__________________________________________")
    ################################################################
    curr_field = rc.fill_theGap_linearLine(curr_field, indeks="EVI", SF_year = 2017)

    ################################################################
    row_pointer = 54 * counter
    output_df[row_pointer: row_pointer+54] = curr_field.values
    counter += 1

# Time Series Linear Regression